In [1]:
from lib import *
from tqdm import tqdm

In [2]:
w = (15, 9, -8, 3, -4)
k = 3
all_R = list(consistent_sets(w, k))

In [3]:
def one_line_max(w):
    return max(range(len(w)), key = lambda i: w[i])

def I_set(w, w_j):
    n = len(w)
    invs = inv(w, 2)
    return {repr(x[0], n) for x in invs if (x[1]%n) == (w_j%n)}

print(f"j = {one_line_max(w)}, w_j = {w[one_line_max(w)]}")
print(f"I = {I_set(w, w[one_line_max(w)])}")

j = 0, w_j = 15
I = {1, 2, 3, 4}


In [4]:
def truncate(w, w_j):
    n = len(w)
    I = I_set(w, w_j)
    v = [x for x in w if x%n in I]
    v = [x - x//n for x in v]
    m = len(v)
    desired_length = m*(m+1)//2
    while sum(v) != desired_length:
        if sum(v) < desired_length:
            v = v[1:] + [v[0]+m]
        else:
            v = [v[-1]-m] + v[:-1] 
    return tuple(v)

w_ = truncate(w, w[one_line_max(w)])
print(f"w' = {w_}")

w' = (3, -3, 12, -2)


In [5]:
def V1(w, k):
    j = one_line_max(w)
    n = len(w)
    x = w[j+1] if j+1 < n else w[0] + n
    y = w[j]
    return set([X for X in inv(w,k) if normalize([X[-2], X[-1]], n) == normalize([x, y], n)])

print(f"V1 = {V1(w, k)}")

V1 = {(3, 4, 10), (2, 19, 25), (3, 9, 15), (1, 4, 10), (2, 9, 15), (2, 4, 10), (1, 9, 15), (2, 14, 20), (1, 14, 20)}


In [6]:
def truncate_set(Xs, n, I):
    ret = set()
    for X in Xs:
        Y = [x for x in X if x%n in I]
        Y = [x - x//n for x in Y]
        ret.add(tuple(Y))
    return ret

print(f"V1' = {truncate_set(V1(w, k), len(w), I_set(w, w[one_line_max(w)]))}")

V1' = {(1, 12), (3, 8), (2, 4), (3, 4), (2, 16), (1, 8), (1, 4), (2, 12), (2, 8)}


In [ ]:
n = 6
found_counterexample = False
for l in tqdm(list(range(15, 36))):
    for w in permutations(n, l):
        k = 3
        n = len(w)

        # ASSUMPTION: w_0 is the largest
        if one_line_max(w) != 0:
            continue

        w_j = w[one_line_max(w)]
        I = I_set(w, w_j)
        all_R = list(consistent_sets(w, k+1))
        w_ = truncate(w, w_j)
        all_R_ = list(consistent_sets(w_, k))

        for R in all_R:
            R = all_R[0]
            Rc = inv(w, k+1) - R
            R_ = truncate_set(Rc & V1(w,k+1), n, I)

            step_1_Xs = set(R_)

            # Rule 2
            step_2_Xs = set()
            for X in inv(w_, k+1):
                PX = packet(X, n-1)
                if PX[0] not in R_ and PX[k-1] in R_:
                    R_.add(PX[k])
                    step_2_Xs.add(PX[k])

            # Rule 3
            step_3_Xs = set()
            step_3_Ys = set()
            for X in quasi_inv(w_, k+1):
                kinv = inv(w_,k)
                PX = packet(X, n-1)
                PX_intersect= set([Y for Y in PX if Y in kinv])
                if PX_intersect == set([PX[-2], PX[-1]]) and PX[-2] in R_ and PX[-1] not in R_:
                    R_.add(PX[-1])
                    step_3_Xs.add(PX[-1])
                    step_3_Ys.add(X)
            
            tw2 = repr(w[1] - w[1] // n, n-1)
            full_inv_Xs = set()
            for U in inv(w_, k+1):
                intersect1 = set(packet(U,n-1)) & (step_1_Xs)
                intersect2 = set(packet(U,n-1)) & (step_2_Xs)
                intersect3 = set(packet(U,n-1)) & (step_3_Xs)
                if repr(U[-1],n-1) != repr(tw2, n-1) and len(intersect1) > 0 and len(intersect2) > 0 and len(intersect3) > 0:
                    found_counterexample = True
                    print(f"Last case counterexample. w = {w}, u={w_}, U = {U}, intersect1={intersect1}, intersect3={intersect3}")
                    print("Step 1 added:", step_1_Xs)
                    print("Step 2 added:", step_2_Xs)
                    print("Step 3 added:", step_3_Xs)
                    break
            
            # if len(step_3_Xs & full_inv_Xs) > 0:
            #     found_counterexample = True
            #     print(f"Part Counterexample w = {w}, u={w_}, l = {l} j = {one_line_max(w)}, k = {k}, R = {R}, R_ = {R_}")
            #     print(step_3_Xs)
            #     print(step_3_Ys)
            #     print(full_inv_Xs)
            #     break
            
            if R_ not in all_R_ and one_line_max(w) == 0:
                found_counterexample = True
                print(f"Counterexample w = {w}, l = {l} j = {one_line_max(w)}, k = {k}, R = {R}, R_ = {R_}")
                print(quasi_inv(w_, k+1))
                break

        if found_counterexample:
            break
    if found_counterexample:
        break

 29%|██▊       | 6/21 [00:03<00:14,  1.04it/s]

In [36]:
w = (10, 8,-3, -6, 6)
w_ = truncate(w, w[one_line_max(w)])
Q = quasi_inv(w_, 4)
I = inv(w_, 4)
(4,6,9,11) in I
# k = 3
# n = len(w)
# R = R = frozenset({(4, 6, 8, 10)})
# Rc = inv(w, k+1) - R
# R_ = truncate_set(Rc & V1(w,k+1), n, I)

# print(f"w' = {w_}")
# print(f"V1 = {V1(w, k+1)}")
# print(f"Inv_3(w) = {inv(w, k+1)}")
# print(f"Rc = {Rc}")
# print(f"R' = {R_}")
# print(f"Inv_2(w') = {inv(w_, k)}")
# print(f"Inv_3(w') = {inv(w_, k+1)}")

True

# Simplified R' Rule

In [37]:
n = 7
found_counterexample = False
for l in tqdm(list(range(18, 24))):
    for w in permutations(n, l):
        k = 3
        n = len(w)

        # ASSUMPTION: w_0 is the largest
        if one_line_max(w) != 0:
            continue

        w_j = w[one_line_max(w)]
        I = I_set(w, w_j)
        all_R = list(consistent_sets(w, k+1))
        w_ = truncate(w, w_j)
        all_R_ = list(consistent_sets(w_, k))

        for R in all_R:
            R = all_R[0]
            Rc = inv(w, k+1) - R
            S = V1(w,k+1)
            for Y in inv(w,k+2) & V1(w,k+2):
                for X in packet(Y,n)[:-1]:
                    S.add(X)
            # for Y in quasi_inv(w, k+2):
            #     for X in set(packet(Y,n)) & inv(w,k+1):
            #         if repr(X[-1],n) == repr(w[0], n):
            #             S.add(X)

            R_ = truncate_set(Rc & S, n, I)

            # Rule 3
            for X in quasi_inv(w_, k+1):
                kinv = inv(w_,k)
                PX = packet(X, n-1)
                PX_intersect= set([Y for Y in PX if Y in kinv])
                if PX_intersect == set([PX[-2], PX[-1]]) and PX[-2] in R_:
                    R_.add(PX[-1])
            if found_counterexample:
                print(R_)
                break
            
            if R_ not in all_R_ and one_line_max(w) == 0:
                found_counterexample = True
                print(f"Counterexample w = {w}, l = {l} j = {one_line_max(w)}, k = {k}, R = {R}, R_ = {R_}")
                print(quasi_inv(w_, k+1))
                break

        if found_counterexample:
            break
    if found_counterexample:
        break


        break

100%|██████████| 6/6 [01:00<00:00, 10.15s/it]


In [25]:
w = (8,1,-2,3)
print(sorted(list(inv(w,2))))
print(sorted(list(inv(w,3))))
print(sorted(list(inv(w,4))))
print(sorted(list(quasi_inv(w,4))))
print(V1(w,3))

[(1, 4), (1, 8), (2, 3), (2, 4), (2, 5), (2, 8), (2, 12), (3, 4), (3, 8)]
[(2, 3, 4), (2, 3, 8), (2, 5, 8), (2, 5, 12)]
[]
[(2, 3, 5, 8)]
{(2, 5, 12)}


In [23]:
u = (-1, 3, 4)
print(sorted(quasi_inv(u, 3)))

[(2, 3, 4)]
